# 数据导入

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
%config InlineBackend.figure_format='retina'

from help import rmspe
from sklearn.metrics import make_scorer
respe_scorer = make_scorer(rmspe, greater_is_better=False)

from sklearn.model_selection import train_test_split
X_reduce = pd.read_csv('data/X_reduce.csv', index_col = 0)
X_predict_reduce = pd.read_csv('data/X_predict_reduce.csv', index_col = 0)
y = pd.read_csv('data/y.csv',header=None ,index_col = 0)
X_train_reduce, X_test_reduce, y_train, y_test = train_test_split(X_reduce, y, random_state=520)

//anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


| No | Model | FileName | Fitting Time | Fitting Size | Train Score(Size) |Train Time| Test Score(Size) | Test Time|
| ------ | ------ | ------ | ------ | ------ | ------ | ------ | ------ | ------ | 
1.1| LinearRegression | Linear_Model.pkl | 1 | Full | 0.47828(Full) | 0 | 0.44169(Full) | 0 |
1.2| LinearRegression | Linear_Model_reduce.pkl | 0 | Full | 0.48339(Full) | 0 | 0.44654(Full) | 0 |
2.1| PolyLinearRegression | Ploy_Res_Model.pkl | 247 | 10000 | 0.34748(10000) | 1 | 9441074912.83404(Full) | 0 |
2.2| PolyLinearRegression | Ploy_Res_Model_reduce.pkl | 9 | Full | 0.45909(Full) | 2 | 0.42838(Full) | 0 |
3.1| RidgeSGDRegressor | Ridge_Sgd_Reg.pkl | 217 | Full | 0.47596(Full) | 0 | 0.43908(Full) | 0 |
3.2| RidgeSGDRegressor | Ridge_Sgd_Reg_reduce.pkl | 161 | Full | 0.48172(Full) | 0 | 0.44459(Full) | 0 |
4.2| KNeighborsRegressor | KNN_grid_search_reduce.pkl | 279 | 40000 | 0.43738(Full) | 50 | 0.40825(Full) | 16 |
|||||
5.1| LinearSVR | Linear_SVR_grid_search.pkl | 8 | 10000 | 0.44014(Full) | 0 | 0.40504(Full) | 0 |
5.2| LinearSVR | Linear_SVR_grid_search_reduce.pkl | 13 | 10000 | 0.44698(Full) | 0 | 0.41071(Full) | 0 |
6.1| KernelSVR | Kernel_SVR_grid_search.pkl | 170 | 6000 | 0.45887(Full) | 142 | 0.42404(Full) | 44 |
6.2| KernelSVR | Kernel_SVR_grid_search_reduce.pkl | 86 | 6000 | 0.45467(Full) | 66 | 0.41992(Full) | 30 |
|||||
7.1| DecisionTreeRegressor | Tree_grid_search.pkl | 694 | Full | 0.18543(Full) | 0 | 0.15738(Full) | 0 |
7.2| DecisionTreeRegressor | Tree_grid_search_reduce.pkl | 353 | Full | 0.19027(Full) | 0 | 0.16006(Full) | 0 |
8.2| RandomForestRegressor | Random_Forest_reduce.pkl | 162 | Full | 0.13618(Full) | 25 | 0.16447(Full) | 5 |
9.2| ExtraTreesRegressor | Extra_Forest_reduce.pkl | 126 | Full | 0.13985(Full) | 22 | 0.16773(Full) | 10 |
|  |  |  |  |  |  |  |  |
10.2| BaggingLinearReg | Bagging_reg_Linear_Model_reduce.pkl | 23 | Full | 0.48347(Full) | 0 | 0.44661(Full) | 0 |
11.2| BaggingLinearSVR | BaggingReg_Linear_SVR_reduce.pkl | 48 | 10000 | 0.44708(Full) | 4 | 0.41076(Full) | 3 |
12.2| BaggingDecisionTree | Bagging_reg_Decision_Tree_reduce.pkl | 91 | Full | 0.18797(Full) | 7 | 0.15228(Full) | 2 |
13.2| AdaDecisionTree | Ada_reg_Decision_Tree_reduce.pkl | 115 | Full | 0.15907(Full) | 5 | 0.15879(Full) | 1 |
14.2| GBRT | GBRT_reg_Decision_Tree_reduce.pkl | 207 | Full |0.15961(Full) | 9 | 0.15607(Full) | 3 |
15.2| XGBoost | XGBoostReg_reduce_log2.pkl | 1917 | Full |0.15860(Full) | 420 | 0.12349(Full) | 16 |
|  |  |  |  |  |  |  |  |

从前面的模型来看，可以发现以下几点规律：

1. 选模型（X.2）与全模型（X.1）相比，测试准确率上几乎相同，但模型训练时间却大幅下降；

2. 树模型在预测效果上远远高于其他模型

# Ensemble

## Voting

In [3]:
DT = joblib.load('Model_Parameter2/Tree_grid_search_reduce.pkl')
RT = joblib.load('Model_Parameter2/Random_Forest_reduce.pkl')
ET = joblib.load('Model_Parameter2/Extra_Forest_reduce.pkl')

In [4]:
from help import Voting, rmspe
rmspe(Voting([DT, RT, ET], data = X_train_reduce), y_train)

0.1521237695208241

In [3]:
from help import Voting, rmspe
rmspe(Voting([DT, RT, ET], data = X_test_reduce), y_test)

0.15885654810060176

In [7]:
submission = pd.read_csv('rossmann-store-sales/sample_submission.csv')
submission['Sales'] = Voting([DT, RT, ET], data = X_predict_reduce)
submission.to_csv('/Users/apple/Documents/Jupyter/Udacity/Rossmann_Store_Sales/submission.csv',
                      index=False)

## Bagging

sklearn.ensemble.BaggingRegressor(base_estimator=None, n_estimators=10, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=None, random_state=None, verbose=0)

In [4]:
from sklearn.ensemble import BaggingRegressor
Base_Model = joblib.load('Model_Parameter2/Tree_grid_search_reduce.pkl').best_estimator_
Bagging_reg_Decision_Tree_reduce = BaggingRegressor(Base_Model, n_estimators=30)

import datetime
starttime = datetime.datetime.now()
print('Fitting model...')
Bagging_reg_Decision_Tree_reduce.fit(X_train_reduce, y_train.values.ravel())
endtime = datetime.datetime.now()
print('  Done!  \n  Using time:',(endtime - starttime).seconds,'sec\n')

Model = Bagging_reg_Decision_Tree_reduce
FileName = 'Bagging_reg_Decision_Tree_reduce'
from help import save_model
save_model(Model, FileName, Best_Model=False)

from help import train_test_score
train_test_score(Model, X=X_reduce, y=y)

from help import submission
submission(Model, Save_File=True)

Fitting model...
  Done!  
  Using time: 91 sec

Saving model...
  The Model have been save in  ['Model_Parameter2/Bagging_reg_Decision_Tree_reduce.pkl']
    PS: To load model, use command: 'Model = joblib.load(FileName)'.
  Using time: 0 sec

Calculating train score...
  Train rmspe score: 0.18797
  Using time: 7 sec

Calculating test score...
  Test rmspe score: 0.15228
  Using time: 2 sec

Predicting submission...
  Done!
  Using time: 0 sec

Saving submission...
  Done!
  PS: To submit file, use command:
    cd /Users/apple/Documents/Jupyter/Udacity/Rossmann_Store_Sales
    kaggle competitions submit -c rossmann-store-sales -f submission.csv -m "Message"


## Adaboost

klearn.ensemble.AdaBoostRegressor(base_estimator=None, n_estimators=50, learning_rate=1.0, loss='linear', random_state=None)

In [53]:
from sklearn.ensemble import AdaBoostRegressor
Base_Model = joblib.load('Model_Parameter2/Tree_grid_search_reduce.pkl').best_estimator_
Ada_reg_Decision_Tree_reduce = AdaBoostRegressor(Base_Model, n_estimators=25,
                                          learning_rate=1)
import datetime
starttime = datetime.datetime.now()
print('Fitting model...')
Ada_reg_Decision_Tree_reduce.fit(X_train_reduce, y_train.values.ravel())
endtime = datetime.datetime.now()
print('  Done!  \n  Using time:',(endtime - starttime).seconds,'sec\n')

Model = Ada_reg_Decision_Tree_reduce
FileName = 'Ada_reg_Decision_Tree_reduce'
from help import save_model
save_model(Model, FileName, Best_Model=False)

from help import train_test_score
train_test_score(Model, X=X_reduce, y=y)

from help import submission
submission(Model, Save_File=True)

Fitting model...
  Done!  
  Using time: 115 sec

Saving model...
  The Model have been save in  ['Model_Parameter2/Ada_reg_Decision_Tree_reduce.pkl']
    PS: To load model, use command: 'Model = joblib.load(FileName)'.
  Using time: 0 sec

Calculating train score...
  Train rmspe score: 0.15907
  Using time: 5 sec

Calculating test score...
  Test rmspe score: 0.15879
  Using time: 1 sec

Predicting submission...
  Done!
  Using time: 0 sec

Saving submission...
  Done!
  PS: To submit file, use command:
    cd /Users/apple/Documents/Jupyter/Udacity/Rossmann_Store_Sales
    kaggle competitions submit -c rossmann-store-sales -f submission.csv -m "Message"


## GBRT

``` Python
sklearn.ensemble.GradientBoostingRegressor(loss='ls', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, init=None, random_state=None, max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, warm_start=False, presort='deprecated', validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
```

In [5]:
from sklearn.ensemble import GradientBoostingRegressor
GBRT_reg_Decision_Tree_reduce = GradientBoostingRegressor(n_estimators=30,
    criterion='mse', max_depth=60, max_features=None,
    max_leaf_nodes=None, min_impurity_decrease=0.0,
    min_impurity_split=None, min_samples_leaf=4,
    min_samples_split=20, min_weight_fraction_leaf=0.0,
    presort=False, random_state=None)

import datetime
starttime = datetime.datetime.now()
print('Fitting model...')
GBRT_reg_Decision_Tree_reduce.fit(X_train_reduce, y_train.values.ravel())
endtime = datetime.datetime.now()
print('  Done!  \n  Using time:',(endtime - starttime).seconds,'sec\n')

Model = GBRT_reg_Decision_Tree_reduce
FileName = 'GBRT_reg_Decision_Tree_reduce'
from help import save_model
save_model(Model, FileName, Best_Model=False)

from help import train_test_score
train_test_score(Model, X=X_reduce, y=y)

from help import submission
submission(Model, Save_File=True)

Fitting model...
  Done!  
  Using time: 153 sec

Saving model...
  The Model have been save in  ['Model_Parameter2/GBRT_reg_Decision_Tree_reduce.pkl']
    PS: To load model, use command: 'Model = joblib.load(FileName)'.
  Using time: 1 sec

Calculating train score...
  Train rmspe score: 0.16818
  Using time: 7 sec

Calculating test score...
  Test rmspe score: 0.15479
  Using time: 2 sec

Predicting submission...
  Done!
  Using time: 0 sec

Saving submission...
  Done!
  PS: To submit file, use command:
    cd /Users/apple/Documents/Jupyter/Udacity/Rossmann_Store_Sales
    kaggle competitions submit -c rossmann-store-sales -f submission.csv -m "Message"


In [6]:
from sklearn.ensemble import GradientBoostingRegressor
GBRT_reg_Decision_Tree_reduce = GradientBoostingRegressor(n_estimators=40,
    criterion='mse', max_depth=60, max_features=None,
    max_leaf_nodes=None, min_impurity_decrease=0.0,
    min_impurity_split=None, min_samples_leaf=4,
    min_samples_split=20, min_weight_fraction_leaf=0.0,
    presort=False, random_state=None)

import datetime
starttime = datetime.datetime.now()
print('Fitting model...')
GBRT_reg_Decision_Tree_reduce.fit(X_train_reduce, y_train.values.ravel())
endtime = datetime.datetime.now()
print('  Done!  \n  Using time:',(endtime - starttime).seconds,'sec\n')

Model = GBRT_reg_Decision_Tree_reduce
FileName = 'GBRT_reg_Decision_Tree_reduce'
from help import save_model
save_model(Model, FileName, Best_Model=False)

from help import train_test_score
train_test_score(Model, X=X_reduce, y=y)

from help import submission
submission(Model, Save_File=True)

Fitting model...
  Done!  
  Using time: 207 sec

Saving model...
  The Model have been save in  ['Model_Parameter2/GBRT_reg_Decision_Tree_reduce.pkl']
    PS: To load model, use command: 'Model = joblib.load(FileName)'.
  Using time: 2 sec

Calculating train score...
  Train rmspe score: 0.15961
  Using time: 9 sec

Calculating test score...
  Test rmspe score: 0.15607
  Using time: 3 sec

Predicting submission...
  Done!
  Using time: 0 sec

Saving submission...
  Done!
  PS: To submit file, use command:
    cd /Users/apple/Documents/Jupyter/Udacity/Rossmann_Store_Sales
    kaggle competitions submit -c rossmann-store-sales -f submission.csv -m "Message"


## [XGBoost](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn)

xgboost.XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=100, verbosity=1, objective='reg:squarederror', booster='gbtree', tree_method='auto', n_jobs=1, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1, colsample_bynode=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0, missing=None, num_parallel_tree=1, importance_type='gain', **kwargs)

``` Python
XGBoostReg = XGBRegressor(
silent=0 ,          # 设置成1则没有运行信息输出，最好是设置为0.是否在运行升级时打印消息。
learning_rate= 0.3, # 如同学习率
min_child_weight=1, # 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
                    #假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
                    #这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。
max_depth=6,        # 构建树的深度，越大越容易过拟合
gamma=0,            # 树的叶子节点上作进一步分区所需的最小损失减少,越大越保守，一般0.1、0.2这样子。
subsample=1,        # 随机采样训练样本 训练实例的子采样比
max_delta_step=0,   #最大增量步长，我们允许每个树的权重估计。
colsample_bytree=1, # 生成树时进行的列采样 
reg_lambda=1,       # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
#reg_alpha=0, # L1 正则项参数
#scale_pos_weight=1, #如果取值大于0的话，在类别样本不平衡的情况下有助于快速收敛。平衡正负权重
#objective= 'multi:softmax', #多分类的问题 指定学习任务和相应的学习目标
#num_class=10, # 类别数，多分类与 multisoftmax 并用
n_estimators=100, #树的个数
seed=1000 #随机种子
#eval_metric= 'auc'
)
```

### reduce

In [28]:
from sklearn import metrics
from xgboost.sklearn import XGBRegressor
XGBoostReg_reduce = XGBRegressor(max_depth=25,
                                 n_estimators=30,
                                 learning_rate=0.1,
                                 min_child_weight=0.7,
                                 n_jobs=4)

import datetime
starttime = datetime.datetime.now()
print('Fitting model...')
XGBoostReg_reduce.fit(X_train_reduce, y_train.values.ravel(),eval_metric=respe_scorer)
endtime = datetime.datetime.now()
print('  Done!  \n  Using time:',(endtime - starttime).seconds,'sec\n')

Model = XGBoostReg_reduce
FileName = 'XGBoostReg_reduce'
from help import save_model
save_model(Model, FileName, Best_Model=False)

from help import train_test_score
train_test_score(Model, X=X_reduce, y=y)

from help import submission
submission(Model, Save_File=True)

Fitting model...
[15:25:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
  Done!  
  Using time: 218 sec

Saving model...
  The Model have been save in  ['Model_Parameter2/XGBoostReg_reduce.pkl']
    PS: To load model, use command: 'Model = joblib.load(FileName)'.
  Using time: 0 sec

Calculating train score...
  Train rmspe score: 0.14676
  Using time: 29 sec

Calculating test score...
  Test rmspe score: 0.14924
  Using time: 13 sec

Predicting submission...
  Done!
  Using time: 1 sec

Saving submission...
  Done!
  PS: To submit file, use command:
    cd /Users/apple/Documents/Jupyter/Udacity/Rossmann_Store_Sales
    kaggle competitions submit -c rossmann-store-sales -f submission.csv -m "Message"


***

### reduce+log

In [2]:
from help import rmspe_log
respe_scorer_log = make_scorer(rmspe_log, greater_is_better=False)
y_log = np.log1p(y)
y_train_log = np.log1p(y_train)
y_test = np.log1p(y_test)

In [ ]:
from sklearn import metrics
from xgboost.sklearn import XGBRegressor
# XGBoostReg_reduce_log = XGBRegressor(max_depth=10,
#                                  n_estimators=30,
#                                  learning_rate=0.1,
#                                  min_child_weight=0.7,
#                                  n_jobs=4)
XGBoostReg_reduce_log = XGBRegressor(max_depth=10,
                                 n_estimators=1000,
                                 subsample=0.9,
                                 colsample_bytree=0.7,
                                 n_jobs=4)

import datetime
starttime = datetime.datetime.now()
print('Fitting model...')
XGBoostReg_reduce_log.fit(X_train_reduce, y_train_log.values.ravel(),eval_metric=respe_scorer)
endtime = datetime.datetime.now()
print('  Done!  \n  Using time:',(endtime - starttime).seconds,'sec\n')

Model = XGBoostReg_reduce_log
FileName = 'XGBoostReg_reduce_log'
from help import save_model
save_model(Model, FileName, Best_Model=False)

from help import train_test_score_log
train_test_score_log(Model, X=X_reduce, y=y_log)

from help import submission_log
submission_log(Model, Save_File=True)

Fitting model...
[16:22:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
  Done!  
  Using time: 4360 sec

Saving model...
  The Model have been save in  ['Model_Parameter2/XGBoostReg_reduce_log.pkl']
    PS: To load model, use command: 'Model = joblib.load(FileName)'.
  Using time: 0 sec

Calculating train score...
  Train rmspe score: 0.15188
  Using time: 1050 sec

Calculating test score...
  Test rmspe score: 0.13288
  Using time: 99 sec

Predicting submission...
  Done!
  Using time: 14 sec

Saving submission...
  Done!
  PS: To submit file, use command:
    cd /Users/apple/Documents/Jupyter/Udacity/Rossmann_Store_Sales
    kaggle competitions submit -c rossmann-store-sales -f submission.csv -m "Message"
[17:54:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Calculating train score...
  Train rmspe score: 0.15188
  Using time: 546 sec

Calculating test score...


### reduce+log+sort

In [2]:
from help import rmspe_log
respe_scorer_log = make_scorer(rmspe_log, greater_is_better=False)
y_log = np.log1p(y)
y_train_log = y_log[6*7*1115:]
y_test_log = y_log[:6*7*1115]

X_train_reduce = X_reduce[6*7*1115:]
X_test_reduce = X_reduce[:6*7*1115]

In [4]:
from sklearn import metrics
from xgboost.sklearn import XGBRegressor
XGBoostReg_reduce_log2 = XGBRegressor(max_depth=10,
                                 n_estimators=1000,
                                 subsample=0.9,
                                 colsample_bytree=0.7,
                                 n_jobs=4)

import datetime
starttime = datetime.datetime.now()
print('Fitting model...')
XGBoostReg_reduce_log2.fit(X_train_reduce, y_train_log.values.ravel(),eval_metric=respe_scorer)
endtime = datetime.datetime.now()
print('  Done!  \n  Using time:',(endtime - starttime).seconds,'sec\n')

Model = XGBoostReg_reduce_log2
FileName = 'XGBoostReg_reduce_log2'
from help import save_model
save_model(Model, FileName, Best_Model=False)

# Model = joblib.load('Model_Parameter2/XGBoostReg_reduce_log2.pkl')
from help import submission_log
submission_log(Model, Save_File=True)

from help import rmspe_log
import datetime
starttime = datetime.datetime.now()
print('Calculating test score...')
print('  Test score',rmspe_log( Model.predict(X_test_reduce), y_test_log))
endtime = datetime.datetime.now()
print('  Done!  \n  Using time:',(endtime - starttime).seconds,'sec\n')

import datetime
starttime = datetime.datetime.now()
print('Calculating train score...')
print('  Trian score',rmspe_log( Model.predict(X_train_reduce), y_train_log))
endtime = datetime.datetime.now()
print('  Done!  \n  Using time:',(endtime - starttime).seconds,'sec\n')

[22:14:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Predicting submission...
  Done!
  Using time: 18 sec

Saving submission...
  Done!
  PS: To submit file, use command:
    cd /Users/apple/Documents/Jupyter/Udacity/Rossmann_Store_Sales
    kaggle competitions submit -c rossmann-store-sales -f submission.csv -m "Message"
Calculating test score...
  Test score 0.1234918654429577
  Done!  
  Using time: 13 sec

Calculating train score...
  Trian score 0.15899582254435649
  Done!  
  Using time: 393 sec

